# Demo: randomly generated graph with multiple layouts.

This notebook uses networkx to generate moderately-sized 1500-node graph with about 3000 edges. This graph is then laid out in several ways:
* Spring layout
* Spectral layout
* Randomly
* In a circular sine wave

The resulting csv files are written to the correct locations in the DataDiVR project for immediate viewing.

The notebook assumes that the DataDiVR executable + supporting files and the github project with this notebook were unpacked/cloned into the same parent directory. This is *strongly recommended*.

In [14]:
import math
import numpy as np
import networkx as nx
import os

DATADIVR_PATH = os.path.realpath(os.path.join(os.path.dirname(os.getcwd()), "DataDiVR"))
LAYOUTS_DIR = os.path.join(DATADIVR_PATH, "viveNet/Content/data/layouts")
LINKS_DIR = os.path.join(DATADIVR_PATH, "viveNet/Content/data/links")

print(DATADIVR_PATH)
print(LAYOUTS_DIR)
print(LINKS_DIR)

/Users/eiofinova/Projects/DataDiVR
/Users/eiofinova/Projects/DataDiVR/viveNet/Content/data/layouts
/Users/eiofinova/Projects/DataDiVR/viveNet/Content/data/links


## Create a random graph with ~3000 edges

In [9]:
G = nx.fast_gnp_random_graph(1500, 0.003)
G.number_of_edges()

3250

## Lay out the nodes using several different algorithms.

Documentation and full list of options on [NetworkX website](https://networkx.github.io/documentation/stable/reference/drawing.html#module-networkx.drawing.layout).

In [10]:
# Will take a few minutes for larger graphs. 
spring_pos = nx.spring_layout(G, dim=3)
spectral_pos =  nx.spectral_layout(G, dim=3)
random_pos = nx.spring_layout(G, dim=3)
circular_pos = nx.circular_layout(G, dim=3)
kamada_pos = nx.kamada_kawai_layout(G, dim=3)

# The circular layout is pretty boring, it's basically just a circle of nodes.
# Let's make it cooler by staggering the nodes along five phases a sine wave lying on a sphere.
def calculate_z_coordinate(point):
    x, y, z = point
    z = 0.5 * math.sin(5 * math.atan(y/x))
    normalizer = math.sqrt(x**2 + y**2 + z**2)
    return(x/normalizer, y/normalizer, z/normalizer)

circular_pos = {k: calculate_z_coordinate(v) for k, v in circular_pos.items()}

## Write the resulting layouts and edges to .csv files for the DataDiVR.

In [15]:
# The edges are [indicated as from_node, to_node, [color]] and thus are the same for each graph.
output_edges = [list(e) for e in G.edges()]

# Output each layout and a correspondingly-named set of edges in the format expected by the DataDiVR.
layout_names = ["spring", "spectral", "random", "circular", "kamada"]
for i, pos in enumerate([spring_pos, spectral_pos, random_pos, circular_pos, kamada_pos]):
    # Add R, G, B, and alpha channels.
    table = np.array([np.concatenate([pos[x], np.array([255, 255, 255, 255, x])], axis=0) for x in pos.keys()])
    # Positions are given in the unit sphere centered at 0, but we need sphere of dim 0.5 centered at 0.5
    # 
    table[:,0:3] = table[:,0:3] * 0.5 + 0.5
    
    np.savetxt(os.path.join(LAYOUTS_DIR, '%s_test.csv' % layout_names[i]), table, delimiter=',',
               fmt=['%5.5f'] * 3 + ['%d'] * 5)
    np.savetxt(os.path.join(LINKS_DIR, '%s_test_edges.csv'% layout_names[i]), output_edges,
               delimiter=',', fmt='%d')
    